In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import re
from sklearn.compose import ColumnTransformer

In [3]:
train_df = pd.read_csv("train.csv")

In [4]:
test_df = pd.read_csv("test.csv")

In [5]:
TOKEN_PATTERN = "[а-яё]+"
stop_words = nltk.corpus.stopwords.words('russian')

In [6]:
X_test = test_df
y_train = train_df["target"].astype(int).values
del train_df['target']
X_train = train_df

In [7]:
def preprocessor(text):
    whitespaced_text = re.sub("[^а-яё!:)(1-9a-zА-ЯA-Z]", ' ', text.lower())
    return whitespaced_text


In [8]:
%%time

from sklearn.linear_model import LogisticRegression

tfidf_model = Pipeline([
    (
        'features',
        ColumnTransformer([
            (
                'url',
                TfidfVectorizer(
                    lowercase=True, ngram_range=(1, 2), preprocessor=preprocessor ,analyzer='char', min_df=5, max_df=0.75
                ),
                'url'
            ),
            (
                'title',
                TfidfVectorizer(
                    lowercase=True, ngram_range=(1, 1), stop_words=stop_words , min_df=3, max_df=0.75
                ),
                'title'
            )
        ])
    ),
    ('clf', LogisticRegression())
])

tfidf_model.fit(X_train, y_train)
y_tf = tfidf_model.predict(X_train)

CPU times: user 15.7 s, sys: 3.7 s, total: 19.4 s
Wall time: 7.97 s


In [9]:
f1_score(y_train, y_tf)

0.9723269283809874

In [49]:
new_dic = {'id':[0], 'url':["uzbeki.com"], 'title' :["туран узбек развлекается порно с армянками"]}
test_df = pd.DataFrame(new_dic)
np.max(tfidf_model.named_steps['clf'].coef_)

21.95752176472656

### Submit

In [ ]:
del X_test["target"]
test_df["target"] = tfidf_model.predict(X_test).astype(bool)
test_df[["id", "target"]].to_csv("dz3.csv", index=False)
!cat ml_baseline.csv | head